In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import pyproj

In [ ]:
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import exposure
from skimage import img_as_float, img_as_ubyte, io

In [ ]:
#from rasterio import features
with rasterio.open('out_raster/index/vari.tif') as dataset:
    vari = dataset.read(1).astype(np.float32)
    meta = dataset.meta
    nodata_value = meta['nodata']  # Extract the nodata value

In [ ]:
 # Create a boolean mask for non-nodata values
valid_mask = vari != nodata_value

In [ ]:
plt.hist(vari[valid_mask].flatten(), bins=100)

In [ ]:
plt.imshow(vari)

In [ ]:
print(np.mean(vari))

In [ ]:
plt.hist(vari.flatten(), bins=100)

In [ ]:
sigma_est = np.mean(estimate_sigma(vari, channel_axis=None))
sigma_est

In [ ]:
denoise = denoise_nl_means(vari, h=3 * sigma_est, fast_mode=True, patch_size=3, patch_distance=6)

In [ ]:
plt.hist(denoise.flatten(), bins=100, range=(-1,1))

In [ ]:
!pip install PyWavelets

In [ ]:
eq = exposure.equalize_adapthist(denoise)

In [ ]:
fig, (a1,a2,a3) = plt.subplots(1,3)
a1.hist(vari.flatten(), bins=100)
a1.set_title('vari')
a2.hist(denoise.flatten(), bins=100)
a2.set_title('denoise')
a3.hist(eq.flatten(), bins=100)
a3.set_title('eq')

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3)
ax1.imshow(vari)
ax2.imshow(denoise)
ax3.imshow(eq)

In [ ]:
from rasterio.transform import from_origin

In [ ]:
# Specify the path to save the output TIFF file
processed_img = 'cleaned/processed_2ha.tif'

# Assuming 'meta' contains the metadata obtained from the original raster file
# Adjust the metadata as necessary for your output
meta['dtype'] = 'float32'  # Define the data type of the output TIFF
transform = from_origin(meta['transform'][2], meta['transform'][5], meta['transform'][0], meta['transform'][3])

with rasterio.open(processed_img, 'w', **meta) as dst:
    dst.write(eq, 1)  # Writing the cleaned array into the TIFF file

In [ ]:
meta

In [ ]:
plt.hist(eq.flatten(), bins=100, range=(-1,1))

In [ ]:
plt.imshow(denoise)

In [ ]:
plt.imshow(eq)

In [ ]:
np.min(eq)

In [ ]:
markers = np.zeros(dataset.shape, dtype=np.float32)

In [ ]:
no_vegetation = (vari > 0.001) & (vari < 0.002)
nitrogen20 = (vari > 0.002) & (vari <= 0.1)
nitrogen40 = (vari > 0.1) & (vari <= 0.2)
nitrogen60 = (vari > 0.2) & (vari <= 0.3)
nitrogen80 = (vari > 0.3) & (vari < 1)
# seg3 = (denoise >=0.2) & (denoise <0.3)
# seg4 = (denoise >=0.3) & (denoise <0.40)
# seg5 = (denoise >=0.4) & (denoise <0.90)

In [ ]:
all_segment = np.zeros((dataset.shape[0], dataset.shape[1], 1))

In [ ]:
plt.hist(denoise.flat, bins=100)

In [ ]:
markers[no_vegetation] = (5)
markers[nitrogen20] = (10)
markers[nitrogen40] = (20)
markers[nitrogen60] = (30)
markers[nitrogen80] = (40)
# all_segment[seg3] = (40)
# all_segment[seg4] = (2)
# all_segment[seg5] = (1)
plt.imshow(markers)

In [ ]:
meta

In [ ]:
out_path = 'out_raster/classifiy_green_6.tif'
# Assuming 'meta' contains the metadata obtained from the original raster file
# Adjust the metadata as necessary for your output
meta['dtype'] = 'float32'  # Define the data type of the output TIFF
transform = from_origin(meta['transform'][2], meta['transform'][5], meta['transform'][0], meta['transform'][3])

# green farm
meta.update(
            dtype=rasterio.float32,  # Update the data type to match the VARI data
                        count=1,  # Only one band for VARI
                        compress='lzw',  # You can choose a compression method if needed
                        tiled=False,
                        blockysize=1,
                        nodata=0.0
)
new_crs_green = pyproj.CRS.from_epsg(32651)
meta.update(crs=new_crs_green)

with rasterio.open(out_path, 'w', **meta) as dst:
    dst.write(markers, 1)  # Writing the cleaned array into the TIFF file

In [ ]:
meta

In [ ]:
plt.imshow(vari)

In [ ]:
from scipy import ndimage as nd
from rasterio.transform import from_origin

In [ ]:
x=5

In [ ]:
seg1_open = nd.binary_opening(nitrogen20, np.ones((x,x)))
seg1_close = nd.binary_closing(seg1_open, np.ones((x,x)))

seg2_open = nd.binary_opening(nitrogen40, np.ones((x,x)))
seg2_close = nd.binary_closing(seg2_open, np.ones((x,x)))
                                                  
seg3_open = nd.binary_opening(nitrogen60, np.ones((x,x)))
seg3_close = nd.binary_closing(seg3_open, np.ones((x,x)))

seg4_open = nd.binary_opening(nitrogen80, np.ones((x,x)))
seg4_close = nd.binary_closing(seg4_open, np.ones((x,x)))

# seg5_open = nd.binary_opening(nitrogen20, np.ones((x,x)))
# seg5_close = nd.binary_closing(seg5_open, np.ones((x,x)))

In [ ]:
cleaned = np.zeros((dataset.shape[0], dataset.shape[1]), dtype=np.float32)

In [ ]:
cleaned[seg1_close] = (0)
cleaned[seg2_close] = (23)
cleaned[seg3_close] = (40)
cleaned[seg4_close] = (2)
#cleaned[seg5_close] = (1)
plt.imshow(cleaned)

In [ ]:
cleaned.shape

In [ ]:
# Specify the path to save the output TIFF file
output_file_cleaned = 'cleaned/clean_2ha.tif'

# Assuming 'meta' contains the metadata obtained from the original raster file
# Adjust the metadata as necessary for your output
meta['dtype'] = 'float32'  # Define the data type of the output TIFF
transform = from_origin(meta['transform'][2], meta['transform'][5], meta['transform'][0], meta['transform'][3])

with rasterio.open(output_file_cleaned, 'w', **meta) as dst:
    dst.write(cleaned, 1)  # Writing the cleaned array into the TIFF file

In [ ]:
from skimage.segmentation import random_walker

In [ ]:
labels = random_walker(eq, markers, beta=10, mode='bf')

In [ ]:
plt.imshow(labels)

In [ ]:
# Specify the path to save the output TIFF file
labels_out = 'labels.tif'

# Assuming 'meta' contains the metadata obtained from the original raster file
# Adjust the metadata as necessary for your output
meta['dtype'] = 'float32'  # Define the data type of the output TIFF
transform = from_origin(meta['transform'][2], meta['transform'][5], meta['transform'][0], meta['transform'][3])

with rasterio.open(labels_out, 'w', **meta) as dst:
    dst.write(labels, 1)  # Writing the cleaned array into the TIFF file